# Resource Allocation Code Demo

The Food Bank of the Southern Tier (FBST) is a member of Feeding America, focused on providing food security for people with limited financial resources, and serves six counties and nearly 4,000 square miles in the New York.  Under normal operations (non COVID times), the Mobile Food Pantry program is among the main activities of the FBST.  The goal of the service is to make nutritious and healthy food more accessible to people in underserved communities.  Even in areas where other agencies provide assistance, clients may not always have access to food due to limited public transportation options, or because those agencies are only open hours or days per work.

Here we do a sample experiment testing out some of the existing and developed algorithms against a randomized heuristic.

In [1]:
import or_suite
import numpy as np

import copy

import os
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd


import gym

In [12]:
# Getting out configuration parameter for the environment
CONFIG = or_suite.envs.env_configs.resource_allocation_foodbank_config(50)
# CONFIG = or_suite.envs.env_configs.resource_allocation_default_config


# Specifying training iteration, epLen, number of episodes, and number of iterations
epLen = CONFIG['num_rounds']
nEps = 1
numIters = 5

# Configuration parameters for running the experiment
DEFAULT_SETTINGS = {'seed': 2, 
                    'recFreq': 1, 
                    'dirPath': '../data/resource/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters, 
                    'saveTrajectory': True, # save trajectory for calculating additional metrics
                    'epLen' : epLen,
                    'render': False,
                    'pickle': False # indicator for pickling final information
                    }

resource_env = gym.make('Resource-v0', config=CONFIG)
mon_env = Monitor(resource_env)


index [ 6 60 23 42 47 30 33 14 56 34 46 68 19 20 48 35 69 62  8 51 66 39 25 31
 59 24 26 40 58 15 21 63 53 41 54 29 50 28  4 45 61  0 22  7 32  5 36 44
 65 10]
mean[[ 30.575  36.69   55.035]
 [  4.3     5.16    7.74 ]
 [ 37.35   44.82   67.23 ]
 [ 19.3    23.16   34.74 ]
 [  8.2     9.84   14.76 ]
 [ 58.975  70.77  106.155]
 [ 45.3    54.36   81.54 ]
 [ 40.05   48.06   72.09 ]
 [ 18.2    21.84   32.76 ]
 [ 34.725  41.67   62.505]
 [ 15.425  18.51   27.765]
 [ 50.225  60.27   90.405]
 [ 31.5    37.8    56.7  ]
 [ 45.25   54.3    81.45 ]
 [  6.2     7.44   11.16 ]
 [  7.525   9.03   13.545]
 [ 44.     52.8    79.2  ]
 [ 37.175  44.61   66.915]
 [ 44.25   53.1    79.65 ]
 [ 14.075  16.89   25.335]
 [ 45.1    54.12   81.18 ]
 [ 28.     33.6    50.4  ]
 [ 44.075  52.89   79.335]
 [ 66.475  79.77  119.655]
 [  5.875   7.05   10.575]
 [ 30.6    36.72   55.08 ]
 [ 36.125  43.35   65.025]
 [  8.45   10.14   15.21 ]
 [  8.575  10.29   15.435]
 [ 39.15   46.98   70.47 ]
 [ 58.225  69.87  104.805]

In [13]:
agents = { # 'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
#  'Random': or_suite.agents.rl.random.randomAgent(),
#  'Equal': or_suite.agents.resource_allocation.equal_allocation.equalAllocationAgent(epLen, CONFIG),
  'FixedThreshold': or_suite.agents.resource_allocation.fixed_threshold.fixedThresholdAgent(epLen, CONFIG),
 'Guardrail-0.5': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.5),
  'Guardrail-0.3': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.3),
#  'Guardrail-0.25': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.25)
}

Mean and variance endomwnets:
[[ 30.56345914   4.30951166  37.42009121  19.28007753   8.17928924
   59.03555576  45.24495368  40.04517948  18.17984522  34.74676068
   15.42736846  50.30707782  31.53914628  45.20418856   6.2012579
    7.52174938  44.01293985  37.19379229  44.33439783  14.05673434
   45.05579214  28.00159061  44.04941751  66.47290044   5.87442489
   30.57470335  36.14010261   8.4171553    8.60506592  39.24326783
   58.10959179  48.1621989   12.78281274  18.90907687   6.09231056
   57.76427429   8.70017062  54.99835707  53.32025262   6.18300409
   64.98634705  50.11126756  41.38326882  42.26182572  36.97813963
   52.84330971  16.71595755   7.23619732  99.24807313  62.77723912]
 [ 36.5607999    5.13400209  45.01317491  23.18947479   9.81978339
   70.70982346  54.39047468  48.01706859  21.85565403  41.63678283
   18.52452734  60.34645863  37.91940533  54.31767788   7.44401085
    9.02156464  52.73547534  44.4485001   53.03624504  16.88348437
   54.19297462  33.56854875  52.

# Step 5: Run Simulations

Run the different heuristics in the environment

In [14]:
import warnings
warnings.simplefilter('ignore')

In [15]:
path_list_line = []
algo_list_line = []
path_list_radar = []
algo_list_radar= []
for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = '../data/resource_'+str(agent)+'/'
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    elif agent == 'AdaQL' or agent == 'Unif QL' or agent == 'AdaMB' or agent == 'Unif MB':
        or_suite.utils.run_single_algo_tune(resource_env, agents[agent], scaling_list, DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(resource_env, agents[agent], DEFAULT_SETTINGS)

    path_list_line.append('../data/resource_'+str(agent))
    algo_list_line.append(str(agent))
    if agent != 'SB PPO':
        path_list_radar.append('../data/resource_'+str(agent)+'/')
        algo_list_radar.append(str(agent))     
        
fig_path = '../figures/'
fig_name = 'resource'+'_line_plot'+'.pdf'
or_suite.plots.plot_line_plots(path_list_line, algo_list_line, fig_path, fig_name, int(nEps / 40)+1)        
        
additional_metric = { 'Efficiency': lambda traj : or_suite.utils.delta_EFFICIENCY(traj, CONFIG),
                    'Hindsight Envy': lambda traj : or_suite.utils.delta_HINDSIGHT_ENVY(traj, CONFIG),
                      'Counterfactual Envy': lambda traj : or_suite.utils.delta_COUNTERFACTUAL_ENVY(traj, CONFIG),
#                     'Budget': lambda traj : or_suite.utils.times_out_of_budget(traj, CONFIG)
#                       'Prop': lambda traj : or_suite.utils.delta_PROP(traj, CONFIG), \
#                       'Exante Envy': lambda traj : or_suite.utils.delta_EXANTE_ENVY(traj, CONFIG)
                    }
fig_name = 'resource'+'_radar_plot'+'.pdf'
or_suite.plots.plot_radar_plots(path_list_radar, algo_list_radar,
fig_path, fig_name,
additional_metric
)

FixedThreshold
index [60 28  7 46 39 49 37  5 67 54 50 20 24 42 56 35 43 61 29 13  3 51 45 44
 68 30 65  1  6 48 40 22 15 14 36 10 55  0  9 25 17 31 66 58 63 52 34 64
 47  4]
mean[[  4.3     5.16    7.74 ]
 [ 55.     66.     99.   ]
 [ 42.125  50.55   75.825]
 [ 15.425  18.51   27.765]
 [ 28.     33.6    50.4  ]
 [  6.5     7.8    11.7  ]
 [  7.775   9.33   13.995]
 [ 52.7    63.24   94.86 ]
 [ 50.625  60.75   91.125]
 [  6.1     7.32   10.98 ]
 [  8.7    10.44   15.66 ]
 [ 45.25   54.3    81.45 ]
 [ 30.6    36.72   55.08 ]
 [ 19.3    23.16   34.74 ]
 [ 18.2    21.84   32.76 ]
 [  7.525   9.03   13.545]
 [  9.95   11.94   17.91 ]
 [ 64.825  77.79  116.685]
 [ 57.65   69.18  103.77 ]
 [ 38.3    45.96   68.94 ]
 [ 42.675  51.21   76.815]
 [ 14.075  16.89   25.335]
 [  6.175   7.41   11.115]
 [  7.25    8.7    13.05 ]
 [ 50.225  60.27   90.405]
 [ 58.975  70.77  106.155]
 [ 99.15  118.98  178.47 ]
 [ 78.65   94.38  141.57 ]
 [ 30.575  36.69   55.035]
 [  6.2     7.44   11.16 ]
 [  8.45   

Action: [[8.70280e-01 9.44220e-01 1.58361e+00 9.70320e-01 5.00000e-04]
 [1.45275e+00 1.35193e+00 5.00000e-04 1.31822e+00 5.00000e-04]
 [6.11800e-01 6.38040e-01 1.18500e+00 6.46050e-01 2.06222e+00]]
Action: [[8.70280e-01 9.44220e-01 1.58361e+00 9.70320e-01 5.00000e-04]
 [1.45275e+00 1.35193e+00 5.00000e-04 1.31822e+00 5.00000e-04]
 [6.11800e-01 6.38040e-01 1.18500e+00 6.46050e-01 2.06222e+00]]
Action: [[8.70280e-01 9.44220e-01 1.58361e+00 9.70320e-01 5.00000e-04]
 [1.45275e+00 1.35193e+00 5.00000e-04 1.31822e+00 5.00000e-04]
 [6.11800e-01 6.38040e-01 1.18500e+00 6.46050e-01 2.06222e+00]]
Action: [[8.70280e-01 9.44220e-01 1.58361e+00 9.70320e-01 5.00000e-04]
 [1.45275e+00 1.35193e+00 5.00000e-04 1.31822e+00 5.00000e-04]
 [6.11800e-01 6.38040e-01 1.18500e+00 6.46050e-01 2.06222e+00]]
Action: [[8.70280e-01 9.44220e-01 1.58361e+00 9.70320e-01 5.00000e-04]
 [1.45275e+00 1.35193e+00 5.00000e-04 1.31822e+00 5.00000e-04]
 [6.11800e-01 6.38040e-01 1.18500e+00 6.46050e-01 2.06222e+00]]
Action: [[

Action: [[8.70280e-01 9.44220e-01 1.58361e+00 9.70320e-01 5.00000e-04]
 [1.45275e+00 1.35193e+00 5.00000e-04 1.31822e+00 5.00000e-04]
 [6.11800e-01 6.38040e-01 1.18500e+00 6.46050e-01 2.06222e+00]]
Action: [[8.70280e-01 9.44220e-01 1.58361e+00 9.70320e-01 5.00000e-04]
 [1.45275e+00 1.35193e+00 5.00000e-04 1.31822e+00 5.00000e-04]
 [6.11800e-01 6.38040e-01 1.18500e+00 6.46050e-01 2.06222e+00]]
index [ 1  6  3 23 45  2 33 69 64 16 60 31 49 54  0  4 41 26 25 65  5 35 18 50
 63 12 61 13 43 15  8 55 59  9 36 10 58 29 39 46 66 40  7 22 11 68 20 44
 24 48]
mean[[ 78.65   94.38  141.57 ]
 [ 30.575  36.69   55.035]
 [ 42.675  51.21   76.815]
 [ 37.35   44.82   67.23 ]
 [  6.175   7.41   11.115]
 [ 69.875  83.85  125.775]
 [ 45.3    54.36   81.54 ]
 [ 44.     52.8    79.2  ]
 [ 53.425  64.11   96.165]
 [ 71.325  85.59  128.385]
 [  4.3     5.16    7.74 ]
 [ 66.475  79.77  119.655]
 [  6.5     7.8    11.7  ]
 [  6.1     7.32   10.98 ]
 [ 50.05   60.06   90.09 ]
 [ 53.35   64.02   96.03 ]
 [ 18.75

index [60 28  7 46 39 49 37  5 67 54 50 20 24 42 56 35 43 61 29 13  3 51 45 44
 68 30 65  1  6 48 40 22 15 14 36 10 55  0  9 25 17 31 66 58 63 52 34 64
 47  4]
mean[[  4.3     5.16    7.74 ]
 [ 55.     66.     99.   ]
 [ 42.125  50.55   75.825]
 [ 15.425  18.51   27.765]
 [ 28.     33.6    50.4  ]
 [  6.5     7.8    11.7  ]
 [  7.775   9.33   13.995]
 [ 52.7    63.24   94.86 ]
 [ 50.625  60.75   91.125]
 [  6.1     7.32   10.98 ]
 [  8.7    10.44   15.66 ]
 [ 45.25   54.3    81.45 ]
 [ 30.6    36.72   55.08 ]
 [ 19.3    23.16   34.74 ]
 [ 18.2    21.84   32.76 ]
 [  7.525   9.03   13.545]
 [  9.95   11.94   17.91 ]
 [ 64.825  77.79  116.685]
 [ 57.65   69.18  103.77 ]
 [ 38.3    45.96   68.94 ]
 [ 42.675  51.21   76.815]
 [ 14.075  16.89   25.335]
 [  6.175   7.41   11.115]
 [  7.25    8.7    13.05 ]
 [ 50.225  60.27   90.405]
 [ 58.975  70.77  106.155]
 [ 99.15  118.98  178.47 ]
 [ 78.65   94.38  141.57 ]
 [ 30.575  36.69   55.035]
 [  6.2     7.44   11.16 ]
 [  8.45   10.14   15.21 ]

index [ 1  6  3 23 45  2 33 69 64 16 60 31 49 54  0  4 41 26 25 65  5 35 18 50
 63 12 61 13 43 15  8 55 59  9 36 10 58 29 39 46 66 40  7 22 11 68 20 44
 24 48]
mean[[ 78.65   94.38  141.57 ]
 [ 30.575  36.69   55.035]
 [ 42.675  51.21   76.815]
 [ 37.35   44.82   67.23 ]
 [  6.175   7.41   11.115]
 [ 69.875  83.85  125.775]
 [ 45.3    54.36   81.54 ]
 [ 44.     52.8    79.2  ]
 [ 53.425  64.11   96.165]
 [ 71.325  85.59  128.385]
 [  4.3     5.16    7.74 ]
 [ 66.475  79.77  119.655]
 [  6.5     7.8    11.7  ]
 [  6.1     7.32   10.98 ]
 [ 50.05   60.06   90.09 ]
 [ 53.35   64.02   96.03 ]
 [ 18.75   22.5    33.75 ]
 [ 36.125  43.35   65.025]
 [ 44.075  52.89   79.335]
 [ 99.15  118.98  178.47 ]
 [ 52.7    63.24   94.86 ]
 [  7.525   9.03   13.545]
 [ 36.5    43.8    65.7  ]
 [  8.7    10.44   15.66 ]
 [ 48.15   57.78   86.67 ]
 [ 65.375  78.45  117.675]
 [ 64.825  77.79  116.685]
 [ 38.3    45.96   68.94 ]
 [  9.95   11.94   17.91 ]
 [ 39.15   46.98   70.47 ]
 [ 44.25   53.1    79.65 ]

index [55 65 22 58  7 66 25 41 47 30 60 45 32 12 43 51 69 49 24 46 54 26 34  0
 68  9 19 63  4 29 16 67 33  2 40 18 15 50 59  5 27 44 13 39 53 14  8 36
 37 52]
mean[[  6.9     8.28   12.42 ]
 [ 99.15  118.98  178.47 ]
 [ 41.5    49.8    74.7  ]
 [  8.575  10.29   15.435]
 [ 42.125  50.55   75.825]
 [ 45.1    54.12   81.18 ]
 [ 44.075  52.89   79.335]
 [ 18.75   22.5    33.75 ]
 [  8.2     9.84   14.76 ]
 [ 58.975  70.77  106.155]
 [  4.3     5.16    7.74 ]
 [  6.175   7.41   11.115]
 [ 37.075  44.49   66.735]
 [ 65.375  78.45  117.675]
 [  9.95   11.94   17.91 ]
 [ 14.075  16.89   25.335]
 [ 44.     52.8    79.2  ]
 [  6.5     7.8    11.7  ]
 [ 30.6    36.72   55.08 ]
 [ 15.425  18.51   27.765]
 [  6.1     7.32   10.98 ]
 [ 36.125  43.35   65.025]
 [ 34.725  41.67   62.505]
 [ 50.05   60.06   90.09 ]
 [ 50.225  60.27   90.405]
 [ 49.325  59.19   88.785]
 [ 31.5    37.8    56.7  ]
 [ 48.15   57.78   86.67 ]
 [ 53.35   64.02   96.03 ]
 [ 57.65   69.18  103.77 ]
 [ 71.325  85.59  128.385]

index [18 54 64 46  5 26 23 63 11 10  1  4 32 15 20 19 61  8 67  7 40 42 53  0
 62 58 12 45 51  3 38 35 30 47 29 52 56 36 66 33 65 14 27 43 39 25 16 49
  2 44]
mean[[ 36.5    43.8    65.7  ]
 [  6.1     7.32   10.98 ]
 [ 53.425  64.11   96.165]
 [ 15.425  18.51   27.765]
 [ 52.7    63.24   94.86 ]
 [ 36.125  43.35   65.025]
 [ 37.35   44.82   67.23 ]
 [ 48.15   57.78   86.67 ]
 [ 34.575  41.49   62.235]
 [ 62.75   75.3   112.95 ]
 [ 78.65   94.38  141.57 ]
 [ 53.35   64.02   96.03 ]
 [ 37.075  44.49   66.735]
 [ 39.15   46.98   70.47 ]
 [ 45.25   54.3    81.45 ]
 [ 31.5    37.8    56.7  ]
 [ 64.825  77.79  116.685]
 [ 44.25   53.1    79.65 ]
 [ 50.625  60.75   91.125]
 [ 42.125  50.55   75.825]
 [  8.45   10.14   15.21 ]
 [ 19.3    23.16   34.74 ]
 [ 12.725  15.27   22.905]
 [ 50.05   60.06   90.09 ]
 [ 37.175  44.61   66.915]
 [  8.575  10.29   15.435]
 [ 65.375  78.45  117.675]
 [  6.175   7.41   11.115]
 [ 14.075  16.89   25.335]
 [ 42.675  51.21   76.815]
 [  6.475   7.77   11.655]

In [16]:
from IPython.display import IFrame
IFrame("../figures/resource_line_plot.pdf", width=600, height=280)

In [17]:
IFrame("../figures/resource_radar_plot.pdf", width=600, height=500)